# Notebook template for executing code using DerivaML.

This notebook should be modified to perform the desired calcuation and *COMITTED* to Gihub prior to execution.

In [ ]:
from deriva_ml import DerivaML, ExecutionConfiguration, DatasetSpec, MLVocab, Execution, RID
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from typing import Any
import os

## Parameters cell


In [ ]:
# These should be set to be the RIDs of input datasets and assets that are downloaded prior to execution.
datasets: list[DatasetSpec] = []
models: list[RID] = []

dry_run = False
hostname = None
catalog_id = None
a = None
b = None
parameters: dict[str, Any] = {"a": a, "b": b}

In [ ]:
# Modify these to your desired server and catalog.
hostname = hostname or os.environ.get("DERIVA_HOST")
catalog_id = catalog_id or os.environ.get("DERIVA_CATALOG")

# Change this line to call the domain specific class derived from DerivaML
deriva_ml = DerivaML(hostname=hostname, catalog_id=catalog_id)

Provide a function that does the desired calculation.  Access to the downloaded datasets and assets is provided via the execution object.  Place any files to be uploaded into the correct locations using the methods provided via the  execution object.

In [ ]:
def do_stuff(execution: Execution):
        print(f" Execution with parameters: {execution.parameters}")
        print(f" Execution with input assets: {[a.as_posix() for a in execution.asset_paths]}")
        print(f"Execution datasets: {execution.datasets}")

In [ ]:
# Create a workflow instance for this specific version of the notebook.  Return an existing workflow if one is found.
deriva_ml.add_term(MLVocab.workflow_type, "Demo Notebook", description="Initial setup of Model Notebook")

# Create an execution instance that will work with the latest version of the input datasets.
config = ExecutionConfiguration(
    datasets=[DatasetSpec(rid=dataset, version=deriva_ml.dataset_version(dataset)) for dataset in datasets],
    assets=models,
    workflow=deriva_ml.create_workflow('demo-workflow', 'Demo Notebook'),
    parameters=parameters,
)
execution = deriva_ml.create_execution(config, dry_run=dry_run)
with execution as e:
    do_stuff(e)

execution.upload_execution_outputs()